In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Wrangling Data Quiz").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/26 13:30:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/26 13:30:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53422)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/Users/ahmetcicekci/Desktop/projects/udacity-data-eng/spa

In [3]:
user_log_df = spark.read.json("sparkify_log_small.json")

Which page did user id "" (empty string) NOT visit?

In [9]:
user_log_df.filter(user_log_df["userId"] == "").select("page").dropDuplicates().show() # visited pages

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



How many female users do we have in the data set?

In [13]:
user_log_df.filter(user_log_df["gender"] == "F").select("userID").dropDuplicates().count()

462

How many songs were played from the most played artist?

In [22]:
from pyspark.sql.functions import desc

user_log_df.groupBy("artist").count().orderBy(desc("count")).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                NULL| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|        Jack Johnson|   36|
|                Muse|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|          Lily Allen|   28|
|Barry Tuckwell/Ac...|   28|
|               Train|   28|
|           Daft Punk|   27|
|          Nickelback|   27|
|           Metallica|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer. (CHALLENGE)

In [46]:
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

user_log_df = user_log_df.filter(user_log_df["userId"] != "")

window = Window.partitionBy('userId').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

ishome = F.udf(lambda x : int(x == 'Home'), IntegerType())

cusum = user_log_df.filter((user_log_df.page == 'NextSong') | (user_log_df.page == 'Home')) \
    .select('userId', 'page', 'ts') \
    .withColumn('homevisit', ishome('page')) \
    .withColumn('period', F.sum('homevisit') \
    .over(window)) # cumulative sum of "homevisit" per user: like a session id between home visits
    
cusum.filter(cusum["userId"] == 100).show(70)

+------+--------+-------------+---------+------+
|userId|    page|           ts|homevisit|period|
+------+--------+-------------+---------+------+
|   100|    Home|1513839673284|        1|     1|
|   100|NextSong|1513776308284|        0|     1|
|   100|    Home|1513776194284|        1|     2|
|   100|NextSong|1513775710284|        0|     2|
|   100|NextSong|1513775556284|        0|     2|
|   100|    Home|1513775431284|        1|     3|
|   100|NextSong|1513750442284|        0|     3|
|   100|NextSong|1513750214284|        0|     3|
+------+--------+-------------+---------+------+



In [49]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .show()

+------+------+-------------+
|userID|period|count(period)|
+------+------+-------------+
|    10|     0|            2|
|   100|     1|            1|
|   100|     2|            2|
|   100|     3|            2|
|  1000|     0|            1|
|  1003|     1|            1|
|  1005|     0|            1|
|  1006|     0|            3|
|  1017|     0|            1|
|  1017|     1|            3|
|  1017|     2|            6|
|  1019|     0|           16|
|  1019|     2|            3|
|  1019|     3|            3|
|  1020|     0|            4|
|  1022|     0|            2|
|  1025|     1|            4|
|  1030|     0|            1|
|  1035|     0|           31|
|  1035|     1|           40|
+------+------+-------------+
only showing top 20 rows



In [50]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}) \
    .show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



24/08/26 16:04:52 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 387830 ms exceeds timeout 120000 ms
24/08/26 16:04:52 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/26 16:04:55 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o